# Data preparation

In [19]:
from dotenv import load_dotenv
import pandas as pd
import os

import tiktoken
import openai

from langdetect import detect, DetectorFactory

In [ ]:
load_dotenv()
openai.api_key = os.getenv("OPENAI_API_KEY")


csv = pd.read_csv('./jobs_data_clean.csv')

df = csv.copy()
df.head()

,Location,Region,EU Member,Schengen Agreement,Google Domain Type,Google Domain Used,Job Title,Company Name,Job Location,Apply Options,Job Description,Work from home,Salary,Schedule type,Qualifications,Job ID,Search Date,Search Query
0,Austria,Europe,True,True,local,google.at,Sandbox Innovation Sdn. Bhd. | Senior Mobile A...,Tideri Jobbörse,Austria,"Trabajo.org - Stellenangebote, Arbeit",We are a dynamic FinTech company headquartered...,NaN,NaN,Full-time,NaN,eyJqb2JfdGl0bGUiOiJTYW5kYm94IElubm92YXRpb24gU2...,2025-01-13 12:33:21 UTC,iOS developer
1,Austria,Europe,True,True,default,google.com,Senior Ios Developer,Pyramid Global Technologies,Austria,"Trabajo.org - Stellenangebote, Arbeit, StudySm...",A minimum of 6+ years of concurrent commercial...,NaN,NaN,Full-time,No degree mentioned,eyJqb2JfdGl0bGUiOiJTZW5pb3IgSW9zIERldmVsb3Blci...,2025-01-13 12:20:43 UTC,iOS developer
2,Austria,Europe,True,True,default,google.com,iOS Developer - Permanent remote,Bluestorm Recruitment by Dazzle,Austria,Jooble,iOS Developer\r\n\r\nOur client is a leading m...,NaN,NaN,Full-time,No degree mentioned,eyJqb2JfdGl0bGUiOiJpT1MgRGV2ZWxvcGVyIC0gUGVybW...,2025-01-13 12:20:43 UTC,iOS developer
3,Austria,Europe,True,True,default,google.com,Sr. Ios Developer,Bykon,Austria,"Trabajo.org - Stellenangebote, Arbeit",In ByKon we're looking for an exceptional Sr.\...,NaN,NaN,Full-time,NaN,eyJqb2JfdGl0bGUiOiJTci4gSW9zIERldmVsb3BlciIsIm...,2025-01-13 12:20:43 UTC,iOS developer
4,Austria,Europe,True,True,default,google.com,Software Engineer/ iOS,Bitpanda,Anywhere,GrabJobs,Who we are\r\n\r\nWe simplify wealth creation....,True,NaN,Full-time,No degree mentioned,eyJqb2JfdGl0bGUiOiJTb2Z0d2FyZSBFbmdpbmVlci8gaU...,2025-01-13 12:20:43 UTC,iOS developer


## Determine the language of vacancies

In [15]:
def count_tokens(text, model="gpt-4"):
    encoding = tiktoken.encoding_for_model(model)
    return len(encoding.encode(text))

text = df.loc[0, "Job Description"]
print("Количество токенов:", count_tokens(text, "gpt-4-turbo"))

Количество токенов: 359


In [11]:
df.loc[0, "Job Description"]

"We are a dynamic FinTech company headquartered in Australia, now expanding our footprint to Malaysia.\r\nSpecializing in innovative payment and remittance solutions.\r\n\r\nWhy Join Us?\r\n\r\nYoung & dynamic workplace & culture (with office recreational amenities provided) Progressive career prospects Well rewarding project contributions Responsibilities:\r\nDevelop, test, and deploy high quality mobile apps for Android/iOS with a focus on performance, scalability, and reliability.\r\nCollaborate with product managers, designers, and developers to deliver tailored FinTech/Payments/Remittance solutions.\r\nImplement responsive design, accessibility, and security best practices.\r\nStay updated on mobile development trends and integrate new tools to improve efficiency.\r\nParticipate in code reviews, debugging, and troubleshooting to maintain code quality.\r\nOptimize mobile development processes for productivity and workflow efficiency.\r\nDesign scalable mobile applications using nat

### `langdetect` library

In [20]:
DetectorFactory.seed = 0  # Фиксируем результат, чтобы не было случайных изменений

def detect_language(text):
    try:
        return detect(text)
    except:
        return "unknown"

# Применяем к колонке Job Description
df["Language langdetect"] = df["Job Description"].apply(detect_language)

In [21]:
df.head()

,Location,Region,EU Member,Schengen Agreement,Google Domain Type,Google Domain Used,Job Title,Company Name,Job Location,Apply Options,Job Description,Work from home,Salary,Schedule type,Qualifications,Job ID,Search Date,Search Query,Language langdetect
0,Austria,Europe,True,True,local,google.at,Sandbox Innovation Sdn. Bhd. | Senior Mobile A...,Tideri Jobbörse,Austria,"Trabajo.org - Stellenangebote, Arbeit",We are a dynamic FinTech company headquartered...,NaN,NaN,Full-time,NaN,eyJqb2JfdGl0bGUiOiJTYW5kYm94IElubm92YXRpb24gU2...,2025-01-13 12:33:21 UTC,iOS developer,en
1,Austria,Europe,True,True,default,google.com,Senior Ios Developer,Pyramid Global Technologies,Austria,"Trabajo.org - Stellenangebote, Arbeit, StudySm...",A minimum of 6+ years of concurrent commercial...,NaN,NaN,Full-time,No degree mentioned,eyJqb2JfdGl0bGUiOiJTZW5pb3IgSW9zIERldmVsb3Blci...,2025-01-13 12:20:43 UTC,iOS developer,en
2,Austria,Europe,True,True,default,google.com,iOS Developer - Permanent remote,Bluestorm Recruitment by Dazzle,Austria,Jooble,iOS Developer\r\n\r\nOur client is a leading m...,NaN,NaN,Full-time,No degree mentioned,eyJqb2JfdGl0bGUiOiJpT1MgRGV2ZWxvcGVyIC0gUGVybW...,2025-01-13 12:20:43 UTC,iOS developer,en
3,Austria,Europe,True,True,default,google.com,Sr. Ios Developer,Bykon,Austria,"Trabajo.org - Stellenangebote, Arbeit",In ByKon we're looking for an exceptional Sr.\...,NaN,NaN,Full-time,NaN,eyJqb2JfdGl0bGUiOiJTci4gSW9zIERldmVsb3BlciIsIm...,2025-01-13 12:20:43 UTC,iOS developer,en
4,Austria,Europe,True,True,default,google.com,Software Engineer/ iOS,Bitpanda,Anywhere,GrabJobs,Who we are\r\n\r\nWe simplify wealth creation....,True,NaN,Full-time,No degree mentioned,eyJqb2JfdGl0bGUiOiJTb2Z0d2FyZSBFbmdpbmVlci8gaU...,2025-01-13 12:20:43 UTC,iOS developer,en


In [22]:
df.columns

Index(['Location', 'Region', 'EU Member', 'Schengen Agreement',
       'Google Domain Type', 'Google Domain Used', 'Job Title', 'Company Name',
       'Job Location', 'Apply Options', 'Job Description', 'Work from home',
       'Salary', 'Schedule type', 'Qualifications', 'Job ID', 'Search Date',
       'Search Query', 'Language langdetect'],
      dtype='object')

In [24]:
df['Language langdetect'].value_counts()

Language langdetect
en       2445
de         93
it         79
es         68
nl         45
fr         36
cs         23
pt         21
pl         19
hu          8
sv          5
ru          4
ro          4
bg          2
fi          2
sk          2
no          1
uk          1
da          1
lv          1
zh-cn       1
hr          1
sl          1
Name: count, dtype: int64